# Scaffold model of Cerebellum

## Connectome notebook

**Please, before running this Notebook, move the mouse on the _'Kernel'_ tab and choose _'Restart & Clear output'_**


Dependencies and output file:

This notebook imports - and requires - three files, namely:

1. **scaffold_params.py**, a configuration file
2. **connectome_functions.py**, with the functions used in the notebook


All are stored in the Network_files directory of our storage. They're not directly available from this notebook; in order to explore them, there are two main options:

1. download them from the storage
2. once the files have been downloaded in the workspace (3rd code cell in this notebook), the user can create a new cell and open a file with the command %load **filename**, where the filename can be either scaffold_params.py or scaffold_functions.py .

**NOTE for user:** The Connectome notebook must be run **after** Cell positioning

This notebook will update the .hdf5 file with a new 'connections' group

This notebook executes all connections in the granular, molecular, Purkinje and DCN layers, together with the connectivity between molecular and granular layers, and Purkinje soma layer and DCN.

For every type of connection, a matrix is generated with two columns: the first one contains the pre-synaptic cells, while in the second column the post-synaptic cells are reported.

Each connection type is calculated and stored independently; in this way the user can either build the entire connectome or choose a single connection or a customised subset.

Some of the connectivity plots genenerated here have been adapted for Fig. 3 of Casali et al (2019) *Reconstruction and Simulation of a Scaffold Model of the Cerebellar Network* Frontiers in Neuroinformatics,  doi.org/10.3389/fninf.2019.00037

In [ ]:
! pip install --upgrade "hbp-service-client==1.0.0" > install.txt

In [ ]:
import os
from ipywidgets import widgets

client = get_hbp_service_client()

# Path to cerebellum collab storage; some files can be downloaded from here
collab_storage_path = '/8124'
bsp_folder = collab_storage_path + '/BSP_usecases/Network_files/'
client.storage.download_file(bsp_folder+'scaffold_params.py', 'scaffold_params.py')
client.storage.download_file(bsp_folder+'connectome_functions.py', 'connectome_functions.py')
#client.storage.download_file(bsp_folder+'to_connectome.txt', 'to_connectome.txt')


In [ ]:
import fnmatch


# Path to local storage 
clb_path = get_collab_storage_path()
files_in_current_storage = client.storage.list(clb_path)
hdf5_files = [files for files in files_in_current_storage
              if fnmatch.fnmatch(files, 'scaffold*.hdf5')]

# If there is a .hdf5 file in the local storage, the user can download it for connectome
#if len(hdf5_files) > 0:
#    pos_file_name = hdf5_files[0]
#    client.storage.download_file(clb_path+'/'+pos_file_name, pos_file_name)
# Otherwise, download a standard .hdf5 positions file from our group collab
#else:
pos_file_name = 'scaffold_full_dcn_400.0x400.0.hdf5'
client.storage.download_file(bsp_folder+pos_file_name, pos_file_name)
    



In [ ]:
# choose whether to calculate connections metrics

def choose_params(opts, val, des, disab):
    return widgets.Dropdown(options=opts, 
                           value = val,
                           description=des,
                           disabled=disab)
metrics_choice = choose_params([True, False], True, 'compute metrics:', False)

metrics_choice


In [ ]:
metrics_on = metrics_choice.value
print(metrics_on)

In [ ]:
# import libraries, variables and functions
import h5py
import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go 
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

from scaffold_params import *

filename = pos_file_name
%store filename > to_connectome.txt

from connectome_functions import *



### Plot function

A general function to do a 3D scatter plot of connectivity data with *Plotly* 

In [ ]:
def plot_connect(pre_name, post_name, conn_mat, geom, pre_color, post_color):
    if geom == 'divergent':
        pre = np.random.randint(np.min(conn_mat[:,0]), np.max(conn_mat[:,0]))
        post = conn_mat[np.in1d(conn_mat[:,0],pre),1]
        pre_position = positions[np.in1d(positions[:,0],pre), 2::]
        post_position = positions[np.in1d(positions[:,0],post), 2::]
        ref_points = np.array([[0.,0.,pre_position[0,1]], [0., volume_base_size[0], pre_position[0,1]],
                      [volume_base_size[0],volume_base_size[1],pre_position[0,1]], [volume_base_size[1], 0.,pre_position[0,1]]])
        traceRef = go.Scatter3d(
                    x = ref_points[:,0], y = ref_points[:,1], z = ref_points[:,2],
                    name = ' ',
                    mode = 'markers',
                    marker=dict(color='rgb(0, 153, 0)', size=1,
                        symbol='circle',
                        line=dict(
                            color='rgba(204, 204, 204)', width=0.01
                        ), opacity=0.1)
                    )
        # Good color for Glomerulus: 'rgb(64, 64, 64)'
        tracePre = go.Scatter3d(
                    x = [pre_position[0,0]], y = [pre_position[0,2]], z = [pre_position[0,1]],
                    mode='markers',
                    name = pre_name,
                    marker=dict(color=pre_color, size=2.5,
                        symbol='circle',
                        line=dict(
                            color='rgba(204, 204, 204)', width=0.5
                        ), opacity=1.)
                    )
        
        # Good color for Golgi cell: 'rgb(0, 153, 0)'
        tracePost = go.Scatter3d(
                    x = post_position[:,0], y = post_position[:,2], z = post_position[:,1],
                    mode='markers',
                    name = post_name,
                    marker=dict(
                        color=post_color, size=8,
                        symbol='circle',
                        line=dict(
                            color='rgba(204, 204, 204)',width=1
                        ), opacity=0.7)
                    )

    else:
        post = np.random.randint(np.min(conn_mat[:,1]), np.max(conn_mat[:,1]))
        pre = conn_mat[np.in1d(conn_mat[:,1],post),0]
        pre_position = positions[np.in1d(positions[:,0],pre), 2::]
        post_position = positions[np.in1d(positions[:,0],post), 2::]
        ref_points = np.array([[0.,0.,pre_position[0,1]], [0., volume_base_size[0], pre_position[0,1]],
                    [volume_base_size[0],volume_base_size[1],pre_position[0,1]], [volume_base_size[1], 0.,pre_position[0,1]]])
        traceRef = go.Scatter3d(
                    x = ref_points[:,0], y = ref_points[:,1], z = ref_points[:,2],
                    name = ' ',
                    mode = 'markers',
                    marker=dict(color='rgb(0, 153, 0)', size=1,
                        symbol='circle',
                        line=dict(
                            color='rgba(204, 204, 204)', width=0.01
                        ), opacity=0.1)
                    )
        tracePre = go.Scatter3d(
                    x = pre_position[:,0], y = pre_position[:,2], z = pre_position[:,1],
                    mode='markers',
                    name = pre_name,
                    marker=dict(color=pre_color, size=2.5,
                        symbol='circle',
                        line=dict(
                            color='rgba(204, 204, 204)', width=0.5
                        ), opacity=1)
                    )

        tracePost = go.Scatter3d(
                    x = [post_position[0,0]], y = [post_position[0,2]], z = [post_position[0,1]],
                    mode='markers',
                    name = post_name,
                    marker=dict(
                        color=post_color, size=8,
                        symbol='circle',
                        line=dict(
                            color='rgba(204, 204, 204)',width=1
                        ), opacity=0.7)
                    )
        
    layout = go.Layout(
            scene = dict(
                aspectmode='data')
            )
    
    data = [traceRef, tracePre, tracePost]
    fig = go.Figure(data=data, layout=layout)
    iplot(fig, filename='connections_example')



## Connectome metrics
Function to compute convergence and divergence ratios for each connection type

In [ ]:
def metrics(pre , post, conn_mat):
    '''pre = presyn neurons
       post = postsyn neurons
       conn_mat = connectivity matrix'''
    # metrics

    # granules ascending axons and Golgi cells dendrites

    # divergence
    count_pre = np.zeros(len(pre))
    for idx,j in enumerate(pre):
        count = (conn_mat[:,0]).__eq__(j)
        count_true=len(np.where(count==True)[0])
        count_pre[idx]=count_true
    print('\nDivergence:')
    print('min:  %.4f' % np.min(count_pre))
    print('max:  %.4f' % np.max(count_pre))
    print('mean: %.4f' % np.mean(count_pre))
    print('std:  %.4f' % np.std(count_pre))

    # convergence
    count_post = np.zeros(len(post))
    for idx,j in enumerate(post):
        count = (conn_mat[:,1]).__eq__(j)
        count_true=len(np.where(count==True)[0])
        count_post[idx]=count_true
    print('\nConvergence:')
    print( 'min: ', np.min(count_post))
    print( 'max: ', np.max(count_post))
    print( 'mean: ', np.mean(count_post))
    print( 'std: ', np.std(count_post))

    # distance between connected cells
    distance = np.zeros(conn_mat.shape[0])
    for idx,index in enumerate(conn_mat):
        i = index.astype(int)
        distance[idx] = np.sqrt((positions[i[0],2]-positions[i[1],2])**2+(positions[i[0],3]-positions[i[1],3])**2+(positions[i[0],4]-positions[i[1],4])**2)
    print( '\nDistance:')
    print( 'min: ', np.min(distance))
    print( 'max: ', np.max(distance))
    print( 'mean: ', np.mean(distance))
    print( 'std: ', np.std(distance))


Cell types:
1. Golgi cells
2. glomeruli
3. granules
4. Purkinje cells
5. basket cells
6. stellate cells
7. DCN glutamatergic cells

In [ ]:
# Open hdf5 file that contains cerebellum positioning and read contents for each cell type
# File reading: the connected cells for each type of connection will be stored in the same file
f = h5py.File(filename, 'r+')

## Functions execution

### Granular layer

In this layers the following connections are performed:
    1. connection between glomeruli and dendrites of granules
    2. connection between glomeruli and basolateral dendrites of Golgi cells
    3. connection between ascending axons of granules and Golgi cells dendrites
    4. connection between parallel fibers and Golgi cells dendrites
    5. connection between the axonal plexus of Golgi cells and glomeruli
    6. connection between the axonal plexus of Golgi cells and granules

In [ ]:
# connections between glomeruli and granules dendrites

glom_grc = connectome_glom_grc(glomeruli, granules, dend_len, n_conn_glom, glom_grc)

if metrics_on:
    metrics(glomeruli_idx, granules_idx, glom_grc)


# Save neurons ID for this connection in the same hdf5 of placement
if 'connections' in f.keys():

    if 'glom_grc' in f['connections'].keys():
        f['connections'].__delitem__('glom_grc')
        f['connections'].create_dataset('glom_grc', data=glom_grc, maxshape=(None, None))
    else:
        f['connections'].create_dataset('glom_grc', data=glom_grc, maxshape=(None, None))

else:
    connections = f.create_group('connections')

    connections.create_dataset('glom_grc', data=glom_grc, maxshape=(None, None))



# DIVERGENCE
plot_connect('Glomeruli', 'Granule cells', glom_grc, 'divergent','rgb(64, 64, 64)' , 'rgb(0, 153, 0)')

# CONVERGENCE
plot_connect('Glomeruli', 'Granule cells', glom_grc, 'convergent','rgb(64, 64, 64)' , 'rgb(0, 153, 0)')

In [ ]:
# connections between glomeruli and Golgi cells basolateral dendrites

glom_goc = connectome_glom_goc(glomeruli, golgicells, r_goc_vol, glom_bd)

if metrics_on:
    metrics(glomeruli_idx, golgi_idx, glom_goc)


# Save data
if 'connections' in f.keys():

    if 'glom_goc' in f['connections'].keys():
        
        f['connections'].__delitem__('glom_goc')
        f['connections'].create_dataset('glom_goc', data=glom_goc, maxshape=(None, None))
    else:
        f['connections'].create_dataset('glom_goc', data=glom_goc, maxshape=(None, None))
else:
    connections = f.create_group('connections')
    
    connections.create_dataset('glom_goc', data=glom_goc, maxshape=(None, None))



# CONVERGENCE
plot_connect('Glomeruli', 'Golgi cells', glom_goc, 'convergent','rgb(64, 64, 64)' , 'rgb(0, 153, 0)')

In [ ]:
# connections between, respectively, granules ascending axons and parallel fibers and Golgi cells dendrites

aa_goc, pf_goc  = connectome_grc_goc(granules, golgicells, r_goc_vol, OoB_value, n_connAA, n_conn_pf, tot_conn, aa_goc, pf_goc)

if metrics_on:
    print('metrics for connectivity between ascending axons and Golgi cells')
    metrics(granules_idx, golgi_idx, aa_goc)
    print ('\nmetrics for connectivity between parallel fibers and Golgi cells')
    metrics(granules_idx, golgi_idx, pf_goc)


# Save data
if 'connections' in f.keys():

    if 'aa_goc' in f['connections'].keys():
        #data=aa_goc
        f['connections'].__delitem__('aa_goc')
        f['connections'].create_dataset('aa_goc', data=aa_goc, maxshape=(None, None))
    else:
        f['connections'].create_dataset('aa_goc', data=aa_goc, maxshape=(None, None))
        
    if 'pf_goc' in f['connections'].keys():
        #data=pf_goc
        f['connections'].__delitem__('pf_goc')
        f['connections'].create_dataset('pf_goc', data=pf_goc, maxshape=(None, None))
    else:
        f['connections'].create_dataset('pf_goc', data=pf_goc, maxshape=(None, None))

else:
    connections = f.create_group('connections')
    
    connections.create_dataset('aa_goc', data=aa_goc, maxshape=(None, None))
    connections.create_dataset('pf_goc', data=pf_goc, maxshape=(None, None))



# AA-GOC CONVERGENCE
plot_connect('Granule cells: ascending axons', 'Golgi cells', aa_goc, 'convergent','rgb(64, 64, 64)' , 'rgb(0, 153, 0)')

# PF-GOC
# DIVERGENCE
plot_connect('Granule cells: parallel fibers', 'Golgi cells', pf_goc, 'divergent','rgb(64, 64, 64)' , 'rgb(0, 153, 0)')
# CONVERGENCE
plot_connect('Granule cells: parallel fibers', 'Golgi cells', pf_goc, 'convergent','rgb(64, 64, 64)' , 'rgb(0, 153, 0)')

In [ ]:
# connections between Golgi cells axon and glomeruli

goc_glom = connectome_goc_glom(glomeruli, golgicells, GoCaxon_x, GoCaxon_y, GoCaxon_z, r_glom, n_conn_goc, OoB_value, axonGOC_glom)

if metrics_on:
    metrics(golgi_idx, glomeruli_idx, goc_glom)

# Save data
if 'connections' in f.keys():

    if 'goc_glom' in f['connections'].keys():
        #data=goc_glom
        f['connections'].__delitem__('goc_glom')
        f['connections'].create_dataset('goc_glom', data=goc_glom, maxshape=(None, None))
    else:
        f['connections'].create_dataset('goc_glom', data=goc_glom, maxshape=(None, None))
        
else:
    connections = f.create_group('connections')
    
    connections.create_dataset('goc_glom', data=goc_glom, maxshape=(None, None))



# DIVERGENCE
plot_connect('Golgi cells', 'Glomeruli', goc_glom, 'divergent','rgb(64, 64, 64)' , 'rgb(0, 153, 0)')

In [ ]:
# connections between Golgi cells axon and granule cells dendrite

goc_grc = connectome_goc_grc(glom_grc, goc_glom)

if metrics_on:
    metrics(golgi_idx, glomeruli_idx, goc_glom)

# Save data
if 'connections' in f.keys():
    if 'goc_grc' in f['connections'].keys():
        f['connections'].__delitem__('goc_grc')
        f['connections'].create_dataset('goc_glom', data=goc_glom, maxshape=(None, None))
else:
    connections = f.create_group('connections')
    connections.create_dataset('goc_glom', data=goc_glom, maxshape=(None, None))



# DIVERGENCE
plot_connect('Golgi cells', 'Granule cells', goc_grc, 'divergent','rgb(64, 64, 64)' , 'rgb(0, 153, 0)')

### Connectivity between granular and molecular layers

Here, the following connections are performed:
    1. connection between ascending axons of granules and the dendritic tree of Purkinje cells
    2. connection between parallel fibers and the dendritic tree of Purkinje cells
    3. connection between parallel fibers and the dendritic tree of basket cells
    4. connection between parallel fibers and the dendritic tree of stellate cells

In [ ]:
# connections between granules ascending axons and Purkinje cells dendritic tree

aa_pc = connectome_aa_pc(granules, purkinjes, x_pc, z_pc, OoB_value, aa_pc)

if metrics_on:
    metrics(granules_idx, purkinjes_idx, aa_pc)


# Save data
if 'connections' in f.keys():

    if 'aa_pc' in f['connections'].keys():
        #data = aa_pc
        f['connections'].__delitem__('aa_pc')
        f['connections'].create_dataset('aa_pc', data=aa_pc, maxshape=(None, None))
    else:
        f['connections'].create_dataset('aa_pc', data=aa_pc, maxshape=(None, None))

else:
    connections = f.create_group('connections')
    
    connections.create_dataset('aa_pc', data=aa_pc, maxshape=(None, None))



# CONVERGENCE
plot_connect('Granule cells: ascending axons', 'Purkinje cells', aa_pc, 'convergent','rgb(64, 64, 64)' , 'rgb(0, 153, 0)')

In [ ]:
# connections between parallel fibers and Purkinje cells dendritic tree

pf_pc = connectome_pf_pc(granules, purkinjes, x_pc, pf_pc)

if metrics_on:
    metrics(granules_idx, purkinjes_idx, pf_pc)


# Save data
if 'connections' in f.keys():

    if 'pf_pc' in f['connections'].keys():
        #data=pf_pc
        f['connections'].__delitem__('pf_pc')
        f['connections'].create_dataset('pf_pc', data=pf_pc, maxshape=(None, None))
    else:
        f['connections'].create_dataset('pf_pc', data=pf_pc, maxshape=(None, None))
        
else:
    connections = f.create_group('connections')
    
    connections.create_dataset('pf_pc', data=pf_pc, maxshape=(None, None))



# DIVERGENCE
plot_connect('Granule cells: parallel fibers', 'Purkinje cells', pf_pc, 'divergent','rgb(64, 64, 64)' , 'rgb(0, 153, 0)')

# CONVERGENCE
plot_connect('Granule cells: parallel fibers', 'Purkinje cells', pf_pc, 'convergent','rgb(64, 64, 64)' , 'rgb(0, 153, 0)')

In [ ]:
# connections between parallel fibers and basket cells dendritic tree

pf_bc = connectome_pf_bc(basketcells, granules, r_sb, h_pf, pf_bc)

if metrics_on:
    metrics(granules_idx, basket_idx, pf_bc)


# Save data
if 'connections' in f.keys():

    if 'pf_bc' in f['connections'].keys():
        #data=pf_bc
        f['connections'].__delitem__('pf_bc')
        f['connections'].create_dataset('pf_bc', data=pf_bc, maxshape=(None, None))
    else:
        f['connections'].create_dataset('pf_bc', data=pf_bc, maxshape=(None, None))
        
else:
    connections = f.create_group('connections')
    
    connections.create_dataset('pf_bc', data=pf_bc, maxshape=(None, None))



# CONVERGENCE
plot_connect('Granule cells: parallel fibers', 'Basket cells', pf_bc, 'convergent','rgb(64, 64, 64)' , 'rgb(0, 153, 0)')

In [ ]:
# connections between parallel fibers and stellate cells dendritic tree

pf_sc = connectome_pf_sc(stellates, granules, r_sb, h_pf, pf_sc)

if metrics_on:
    metrics(granules_idx, stellates_idx, pf_sc)


# Save data
if 'connections' in f.keys():

    if 'pf_sc' in f['connections'].keys():
        #data=pf_sc
        f['connections'].__delitem__('pf_sc')
        f['connections'].create_dataset('pf_sc', data=pf_sc, maxshape=(None, None))
    else:
        f['connections'].create_dataset('pf_sc', data=pf_sc, maxshape=(None, None))
        
else:
    connections = f.create_group('connections')
    
    connections.create_dataset('pf_sc', data=pf_sc, maxshape=(None, None))



# CONVERGENCE
plot_connect('Granule cells: parallel fibers', 'Stellate cells', pf_sc, 'convergent','rgb(64, 64, 64)' , 'rgb(0, 153, 0)')

### Molecular layer

In this layer, the following connections are reported:
    1. connection between stellate cells and the dendritic tree of Purkinje cells
    2. connection between basket cells and the dendritic tree of Purkinje cells
    3. gap junctions of stellate cells
    4. gap junctions of basket cells

In [ ]:
# connections between, respectively, stellate and basket cells and Purkinje dendritic tree

sc_pc, bc_pc = connectome_sc_bc_pc(basketcells, stellates, purkinjes, distx, distz, conv, sc_pc, bc_pc)

if metrics_on:
    print('metrics for connectivity between stellate cells and Purkinje cells')
    metrics(stellates_idx, purkinjes_idx, sc_pc)
    print('\nmetrics for connectivity between basket cells and Purkinje cells')
    metrics(basket_idx, purkinjes_idx, bc_pc)


# Save data
if 'connections' in f.keys():

    if 'sc_pc' in f['connections'].keys():
        data=sc_pc
        f['connections'].__delitem__('sc_pc')
        f['connections'].create_dataset('sc_pc', data=sc_pc, maxshape=(None, None))
    else:
        f['connections'].create_dataset('sc_pc', data=sc_pc, maxshape=(None, None))

    if 'bc_pc' in f['connections'].keys():
        #data=bc_pc
        f['connections'].__delitem__('bc_pc')
        f['connections'].create_dataset('bc_pc', data=bc_pc, maxshape=(None, None))
    else:
        f['connections'].create_dataset('bc_pc', data=bc_pc, maxshape=(None, None))
        
else:
    connections = f.create_group('connections')
    
    connections.create_dataset('sc_pc', data=sc_pc, maxshape=(None, None))
    connections.create_dataset('bc_pc', data=bc_pc, maxshape=(None, None))



# SC-PC CONVERGENCE
plot_connect('Stellate cells', 'Purkinje cells', sc_pc, 'convergent','rgb(64, 64, 64)' , 'rgb(0, 153, 0)')

# BC-PC CONVERGENCE
plot_connect('Basket cells', 'Purkinje cells', bc_pc, 'convergent','rgb(64, 64, 64)' , 'rgb(0, 153, 0)')

In [ ]:
# gap junctions of stellate cells

gj_sc = gap_junctions_sc(stellates, d_xy, d_z, dc_gj, gj_sc)

if metrics_on:
    metrics(stellates_idx, stellates_idx, gj_sc)


# Save data
if 'connections' in f.keys():

    if 'gj_sc' in f['connections'].keys():
        #data=gj_sc
        f['connections'].__delitem__('gj_sc')
        f['connections'].create_dataset('gj_sc', data=gj_sc, maxshape=(None, None))
    else:
        f['connections'].create_dataset('gj_sc', data=gj_sc, maxshape=(None, None))
        
else:
    connections = f.create_group('connections')
    
    connections.create_dataset('gj_sc', data=gj_sc, maxshape=(None, None))



# DIVERGENCE
plot_connect('Stellate cells', 'Stellate cells', gj_sc, 'divergent','rgb(64, 64, 64)' , 'rgb(0, 153, 0)')

# CONVERGENCE
plot_connect('Stellate cells', 'Stellate cells', gj_sc, 'convergent','rgb(64, 64, 64)' , 'rgb(0, 153, 0)')

In [ ]:
# gap junctions of basket cells

gj_bc = gap_junctions_bc(basketcells, d_xy, d_z, dc_gj, gj_bc)

if metrics_on:
    metrics(basket_idx, basket_idx, gj_bc)


# Save data
if 'connections' in f.keys():

    if 'gj_bc' in f['connections'].keys():
        #data=gj_bc
        f['connections'].__delitem__('gj_bc')
        f['connections'].create_dataset('gj_bc', data=gj_bc, maxshape=(None, None))
    else:
        f['connections'].create_dataset('gj_bc', data=gj_bc, maxshape=(None, None))
        
else:
    connections = f.create_group('connections')
    
    connections.create_dataset('gj_bc', data=gj_bc, maxshape=(None, None))



# DIVERGENCE
plot_connect('Basket cells', 'Basket cells', gj_bc, 'divergent','rgb(64, 64, 64)' , 'rgb(0, 153, 0)')

Connectivity between layer of Purkinje cells soma and DCN layer

Here, it is reported the connection between the axon of Purkinje cells and the dendritic tree of glutamatergic cells of DCN

In [ ]:
# connectivity between Purkinje cells axon and glutamatergic DCN cells

pc_dcn = connectome_pc_dcn(purkinjes, dcn_glut, div_pc, dend_tree_coeff, pc_dcn)

#if metrics_on:
#    metrics(purkinjes_idx, dcn_idx, pc_dcn)


# Save data
if 'connections' in f.keys():

    if 'pc_dcn' in f['connections'].keys():
        #data=pc_dcn
        f['connections'].__delitem__('pc_dcn')
        f['connections'].create_dataset('pc_dcn', data=pc_dcn, maxshape=(None, None))
    else:
        f['connections'].create_dataset('pc_dcn', data=pc_dcn, maxshape=(None, None))
        
else:
    connections = f.create_group('connections')
    
    connections.create_dataset('pc_dcn', data=pc_dcn, maxshape=(None, None))



# DIVERGENCE
plot_connect('Purkinje cells', 'DCN cells', pc_dcn, 'divergent','rgb(64, 64, 64)' , 'rgb(0, 153, 0)')

# CONVERGENCE
plot_connect('Purkinje cells', 'DCN cells', pc_dcn, 'convergent','rgb(64, 64, 64)' , 'rgb(0, 153, 0)')

In [ ]:
f.close()

The following two cells plot an example of connection, in this case the one between glomeruli and Golgi cells basolateral dendrites.

The first plot represents the divergence: it randomly selects one presynaptic glomerulus and shows all the postsynaptic cells connected to it. In particular, the figure represents all somas with their different radii.

The second function shows the convergence, by randomly selecting one postsynaptic Golgi cell and displaying all the presynaptic cells connected, with their smaller radius. Here, it can be observed that the glomeruli are correctly placed all under or at the same level of the Golgi cell soma.

Only method by now that allows to 'update' .hdf5 file in storage

In [ ]:
nfilename = 'connected_{}'.format(filename)
print(nfilename)
nfile = h5py.File(nfilename, 'w')
nfile.create_dataset('positions', data=positions)
nfile.create_group('connections')
nfile['connections'].create_dataset('glom_grc', data=glom_grc)
nfile['connections'].create_dataset('glom_goc', data=glom_goc)
nfile['connections'].create_dataset('aa_goc', data=aa_goc)
nfile['connections'].create_dataset('pf_goc', data=pf_goc)
nfile['connections'].create_dataset('goc_glom', data=goc_glom)
nfile['connections'].create_dataset('goc_grc', data=goc_grc)
nfile['connections'].create_dataset('aa_pc', data=aa_pc)
nfile['connections'].create_dataset('pf_pc', data=pf_pc)
nfile['connections'].create_dataset('pf_bc', data=pf_bc)
nfile['connections'].create_dataset('pf_sc', data=pf_sc)
nfile['connections'].create_dataset('bc_pc', data=bc_pc)
nfile['connections'].create_dataset('sc_pc', data=sc_pc)
nfile['connections'].create_dataset('gj_bc', data=gj_bc)
nfile['connections'].create_dataset('gj_sc', data=gj_sc)
nfile['connections'].create_dataset('pc_dcn', data=pc_dcn)
nfile.close()

nfile_to_storage = clb_path+'/'+nfilename
if client.storage.exists(nfile_to_storage):
    client.storage.delete(nfile_to_storage)
client.storage.upload_file(nfilename, nfile_to_storage, 'text/plain')

print('Positions and connections data are stored in\n{}'.format(nfile_to_storage))



